In [10]:
import torch, torchvision
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms

import numpy as np

# monacoBERT + Grad-CAM

In [1]:
class ActivationsAndGradients:
    """ Class for extracting activations and
    registering gradients from targetted intermediate layers 
    
    ref : https://github.com/jacobgil/pytorch-grad-cam/blob/a701308935d7bc18ec0659eddc95fd6d3e315fd2/pytorch_grad_cam/activations_and_gradients.py#L1
    """
    def __init__(self, model, target_layers):
        self.model = model
        self.gradients = []
        self.activations = []
        self.handles = []
        for target_layer in target_layers:
            self.handles.append(
                target_layer.register_forward_hook(self.save_activation))
            # Because of https://github.com/pytorch/pytorch/issues/61519,
            # we don't use backward hook to record gradients.
            self.handles.append(
                target_layer.register_forward_hook(self.save_gradient))

    def save_activation(self, module, input, output):
        activation = output
        self.activations.append(activation.cpu().detach())

    def save_gradient(self, module, input, output):
        if not hasattr(output, "requires_grad") or not output.requires_grad:
            # You can only register hooks on tensor requires grad.
            return

        # Gradients are computed in reverse order
        def _store_grad(grad):
            self.gradients = [grad.cpu().detach()] + self.gradients

        output.register_hook(_store_grad)

    def __call__(self, x):
        self.gradients = []
        self.activations = []
        return self.model(x)

    def release(self):
        for handle in self.handles:
            handle.remove()

In [2]:
# config
import torch
import torch.nn as nn

# hyperparameters
gpu_id = 0 if torch.cuda.is_available() else -1
device = torch.device('cuda:1')
max_seq_len = 100
train_ratio = .8
valid_ratio = .1
hidden_size = 512
output_size = 1
num_head = 8
num_encoder = 12
use_leakyrelu = True
dropout_p = .1
learning_rate = 0.001
grad_acc = True
grad_acc_iter = 8

batch_size = 128
n_epochs = 2

In [3]:
# dataloader
from dataloaders.assist2009_pid_diff_loader import ASSIST2009_PID_DIFF
from utils import pid_diff_collate_fn
from torch.utils.data import DataLoader, random_split

dataset = ASSIST2009_PID_DIFF(max_seq_len)
num_q = dataset.num_q
num_r = dataset.num_r
num_pid = dataset.num_pid
num_diff = dataset.num_diff
collate = pid_diff_collate_fn

train_size = int( len(dataset) * train_ratio * (1 - valid_ratio))
valid_size = int( len(dataset) * train_ratio * valid_ratio)
test_size = len(dataset) - (train_size + valid_size)

train_dataset, valid_dataset, test_dataset = random_split(
    dataset, [ train_size, valid_size, test_size ]
    )

train_loader = DataLoader(
    train_dataset,
    batch_size = batch_size,
    shuffle = True, # train_loader use shuffle
    collate_fn = collate
)
valid_loader = DataLoader(
    valid_dataset,
    batch_size = batch_size,
    shuffle = False, # valid_loader don't use shuffle
    collate_fn = collate
)
test_loader = DataLoader(
    test_dataset,
    batch_size = batch_size,
    shuffle = False, # test_loader don't use shuffle
    collate_fn = collate
)

In [4]:
%%time
from models.visualizer_monacobert_gradcam import MonaConvBert4ktPlusDiffGradCAM

model = MonaConvBert4ktPlusDiffGradCAM(
    num_q=num_q,
    num_r=num_r,
    num_pid=num_pid,
    num_diff=num_diff,
    hidden_size=hidden_size,
    output_size=output_size,
    num_head=16, # TODO : 16 인데 실제로는 8 임, head_ratio 확인
    num_encoder=num_encoder,
    max_seq_len=max_seq_len,
    device=device,
    use_leakyrelu=use_leakyrelu,
    dropout_p=dropout_p
).to(device)

model_path = '/root/BiDKT/model_records/assist09.pth'
model_dict = torch.load(model_path)
model.load_state_dict(model_dict['model'])

<All keys matched successfully>

In [5]:
target_layers = []
for encoder in model.encoder:
    target_layers.extend([encoder.attn.attn_hook_helper, 
                          encoder.attn.conv_hook_helper])
    
grad_attn_layers = {}
grad_conv_layers = {}
for layer_idx in range(num_encoder):
    for head_idx in range(num_head):
        grad_attn_layers[f'{layer_idx}-{head_idx}'] = []
        grad_conv_layers[f'{layer_idx}-{head_idx}'] = []

In [6]:
act_and_grad = ActivationsAndGradients(
    model = model, 
    target_layers = target_layers,
)

In [7]:
from torch.nn.functional import binary_cross_entropy
crit = binary_cross_entropy

In [11]:
def Mlm4BertTest(r_seqs, mask_seqs):
    #|r_seqs| = (bs, n)

    mlm_r_seqs = []
    mlm_idxs = []

    for r_seq, mask_seq in zip(r_seqs, mask_seqs):
        r_len = r_seq.size(0)

        real_r_seq = torch.masked_select(r_seq, mask_seq).cpu()
        real_r_seq_len = real_r_seq.size(0)

        # last index of real_r_seq
        mlm_idx = real_r_seq_len - 1
        # last index get a <MASK>, <MASK> is 2
        real_r_seq[mlm_idx] = 2

        pad_len = r_len - real_r_seq_len
        pad_seq = torch.full((1, pad_len), 3).squeeze(0) # <PAD> is 3
        pad_r_seq = torch.cat((real_r_seq, pad_seq), dim=-1)
        mlm_r_seqs.append(pad_r_seq)

        mlm_zeros = np.zeros(shape=(r_len, ))
        mlm_zeros[mlm_idx] = 1
        mlm_idxs.append(mlm_zeros)

    mlm_r_seqs = torch.stack(mlm_r_seqs)
    mlm_idxs = torch.BoolTensor(mlm_idxs)

    return mlm_r_seqs, mlm_idxs
    # |mlm_r_seqs| = (bs, n)
    # |mask_seqs| = (bs, n)

In [12]:
from tqdm import tqdm

model.eval()
for batch_idx, data in tqdm(enumerate(test_loader)):
    q_seqs, r_seqs, pid_seqs, diff_seqs, mask_seqs = data

    q_seqs = q_seqs.to(device)
    r_seqs = r_seqs.to(device)
    pid_seqs = pid_seqs.to(device)
    diff_seqs = diff_seqs.to(device)
    mask_seqs = mask_seqs.to(device)

    real_seqs = r_seqs.clone()

    mlm_r_seqs, mlm_idxs = Mlm4BertTest(r_seqs, mask_seqs)

    mlm_r_seqs = mlm_r_seqs.to(device)
    mlm_idxs = mlm_idxs.to(device)

    y_hat = model(
        q_seqs.long(),
        mlm_r_seqs.long(),
        pid_seqs.long(),
        diff_seqs.long(),
        mask_seqs.long()
    )

    y_hat = y_hat.squeeze()

    y_hat = torch.masked_select(y_hat, mlm_idxs)
    correct = torch.masked_select(real_seqs, mlm_idxs)

    loss = crit(y_hat, correct)
    loss.backward()
    
#     for grad in act_and_grad.gradients:
#         print(grad.shape)
    
#     act_and_grad.gradients = []
#     act_and_grad.activations = []

0it [00:00, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)
9it [00:07,  1.15it/s]


In [ ]:
relu = nn.ReLU()

def _batch_to_sample(data):
    batch = [torch.stack([
        q_seqs.type(torch.long), 
        r_seqs.type(torch.long), 
        pid_seqs.type(torch.long), 
        diff_seqs.type(torch.long), 
        mask_seqs.type(torch.long)], dim = 1) for q_seqs, r_seqs, pid_seqs, diff_seqs, mask_seqs in zip(*data)]
    return batch

for batch_idx, data in tqdm(enumerate(test_loader)):
    batch = _batch_to_sample(data)
    if len(act_and_grad.gradients[2*batch_idx:2*batch_idx+24]) == 24:
        gradients = act_and_grad.gradients[2*batch_idx:2*batch_idx+24]
    else:
        gradients = act_and_grad.gradients[2*batch_idx:]
    
    for seq_idx, seq in enumerate(batch):
        for sample_idx, sample in enumerate(seq):
            q    = int(sample[0])
            r    = int(sample[1])
            pid  = int(sample[2])
            diff = int(sample[3])
            is_mask = bool(sample[4])                

            if is_mask is True:
                for encoder_idx in range(num_encoder):
                    for head_idx in range(num_head):
                        grad_attn_layers[f'{layer_idx}-{head_idx}'] += \
                        [relu(gradients[2*encoder_idx])[seq_idx, sample_idx, head_idx, ...]]
                        grad_conv_layers[f'{layer_idx}-{head_idx}'] += \
                        [relu(gradients[2*encoder_idx+1][seq_idx, sample_idx, head_idx, ...])]

0it [00:00, ?it/s]

In [ ]:
1

In [ ]:
print(activation(act_and_grad.gradients[0]).mean())

In [166]:
act_and_grad.gradients[0].shape

torch.Size([32, 100, 8, 32])

In [125]:
act_and_grad.activations[0].shape

torch.Size([32, 100, 8, 32])

384

In [ ]:
def _save_grad_per_sample(batch, act_and_grad, grad_attn_layers, grad_conv_layers):
    for seq_idx, seq in enumerate(batch):
        for sample_idx, sample in enumerate(seq):
            q    = int(sample[0])
            r    = int(sample[1])
            pid  = int(sample[2])
            diff = int(sample[3])
            is_mask = bool(sample[4])                

            print(activation(act_and_grad.gradients[0]).mean())

            if is_mask is True:
                for encoder_idx in range(num_encoder):
                    for head_idx in range(num_head):
                        act_and_grad.gradients[2*encoder_idx][seq_idx][head_idx]
                        act_and_grad.gradients[2*encoder_idx+1][seq_idx]
                        # save 
                        grad_attn_layers[]
                        grad_attn_layers[f'-{head}']
                        grad_conv_layers

                        # save filter
                        self.filter_dict_q[q][encoder_idx]       += [filter_layers[seq_idx, sample_idx, encoder_idx, ...]]
                        self.filter_dict_pid[pid][encoder_idx]   += [filter_layers[seq_idx, sample_idx, encoder_idx, ...]]
                        self.filter_dict_diff[diff][encoder_idx] += [filter_layers[seq_idx, sample_idx, encoder_idx, ...]]

                        # save conv value vector
                        self.conv_value_dict_q[q][encoder_idx]       += [value_layers[seq_idx, sample_idx, encoder_idx, ...]]
                        self.conv_value_dict_pid[pid][encoder_idx]   += [value_layers[seq_idx, sample_idx, encoder_idx, ...]]
                        self.conv_value_dict_diff[diff][encoder_idx] += [value_layers[seq_idx, sample_idx, encoder_idx, ...]]

                        # save attn
                        for head_idx in range(model.encoder[0].attn.num_attention_heads):
                            self.attn_dict[head_idx][encoder_idx] += [attn_layers[seq_idx, encoder_idx, head_idx, ...]]

In [156]:
batch = _split_batch(data)

In [ ]:
def _save_blackbox(self, data):
    '''save attn, filter, conv_value to self.~
    '''

    def _hook_blackbox(attn):
        '''get attn, filter, conv_value from self.model.encoder.attn
        attn_scores : [bs, head_idx, num_seq, num_seq]
        conv_values : [bs, num_seq, head_idx, hidden_dim]
        conv_filter : [bs, num_seq, head_idx, kernel_size]
        '''
        attn_scores = attn.attn_scores 
        conv_values = rearrange(
            attn.conv_value_vectors.detach().clone(), 
            'bs (h_idx h_size) n 1 -> bs n h_idx h_size', h_idx = 8)
        conv_filter = rearrange(
            attn.filters.detach().clone(), 
            '(bs n h) k 1 -> bs n h k', n = 100, h = 8)
        return {'attn' : attn_scores, 'value' : conv_values, 'filter' : conv_filter}

    # organize data, hook to easily get the attr
    batch = [torch.stack([q_seqs.type(torch.long), 
                          r_seqs.type(torch.long), 
                          pid_seqs.type(torch.long), 
                          diff_seqs.type(torch.long), 
                          mask_seqs.type(torch.long)], dim = 1) for q_seqs, r_seqs, pid_seqs, diff_seqs, mask_seqs in zip(*data)]

    attn_list = [self.model.encoder[encoder_idx].attn for encoder_idx in range(num_encoder)]
    hook_list = list(map(_hook_blackbox, attn_list))

    attn_layers = torch.stack([hook['attn'] for hook in hook_list], dim = 1) # [bs, layer_idx, head_idx, num_seq, num_seq]
    value_layers = torch.stack([hook['value'] for hook in hook_list], dim = 2) # [bs, num_seq, layer_idx, head_idx, hidden_dim]
    filter_layers = torch.stack([hook['filter'] for hook in hook_list], dim = 2) # [bs, num_seq, layer_idx, head_idx, kernel_sizehook_list

    # save hook to self.~
    for seq_idx, seq in enumerate(batch):
        for sample_idx, sample in enumerate(seq):
            q    = int(sample[0])
            r    = int(sample[1])
            pid  = int(sample[2])
            diff = int(sample[3])
            is_mask = bool(sample[4])                

            if is_mask is True:
                for encoder_idx in range(num_encoder):
                    # save filter
                    self.filter_dict_q[q][encoder_idx]       += [filter_layers[seq_idx, sample_idx, encoder_idx, ...]]
                    self.filter_dict_pid[pid][encoder_idx]   += [filter_layers[seq_idx, sample_idx, encoder_idx, ...]]
                    self.filter_dict_diff[diff][encoder_idx] += [filter_layers[seq_idx, sample_idx, encoder_idx, ...]]

                    # save conv value vector
                    self.conv_value_dict_q[q][encoder_idx]       += [value_layers[seq_idx, sample_idx, encoder_idx, ...]]
                    self.conv_value_dict_pid[pid][encoder_idx]   += [value_layers[seq_idx, sample_idx, encoder_idx, ...]]
                    self.conv_value_dict_diff[diff][encoder_idx] += [value_layers[seq_idx, sample_idx, encoder_idx, ...]]

                    # save attn
                    for head_idx in range(model.encoder[0].attn.num_attention_heads):
                        self.attn_dict[head_idx][encoder_idx] += [attn_layers[seq_idx, encoder_idx, head_idx, ...]]

# MNIST + simple CNN example

In [43]:
import torch, torchvision
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms

import numpy as np

In [44]:
batch_size = 128
num_epochs = 2
device = torch.device('cpu')

In [64]:
# simple cnn
class CNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 10, kernel_size=5),
            nn.MaxPool2d(2),
            nn.ReLU(),
            nn.Conv2d(10, 20, kernel_size=5),
            nn.Dropout(),
            nn.MaxPool2d(2),
            nn.ReLU(),
        )
        
        self.fc_layers = nn.Sequential(
            nn.Linear(320, 50),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(50, 10),
            nn.Softmax(dim=1)
        )
        
        self.hook_helper = HookHelper()

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(-1, 320)
        x = self.hook_helper(x)
        x = self.fc_layers(x)
        return x
    
class HookHelper(nn.Module):
    '''custom Module wrapper for register_forward_hook
    '''
    def forward(self, x):
        return x

In [65]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('mnist_data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor()
                   ])),
    batch_size=batch_size, shuffle=True)

In [66]:
for batch in train_loader:
    break

In [67]:
cnn = CNN()

In [73]:
act_and_grad = ActivationsAndGradients(
    model = cnn, 
    target_layers = [cnn.conv_layers[2], cnn.conv_layers[6], cnn.hook_helper]
)

In [74]:
output = act_and_grad(batch[0])

In [52]:
class ActivationsAndGradients:
    """ Class for extracting activations and
    registering gradients from targetted intermediate layers """

    def __init__(self, model, target_layers):
        self.model = model
        self.gradients = []
        self.activations = []
        self.handles = []
        for target_layer in target_layers:
            self.handles.append(
                target_layer.register_forward_hook(self.save_activation))
            # Because of https://github.com/pytorch/pytorch/issues/61519,
            # we don't use backward hook to record gradients.
            self.handles.append(
                target_layer.register_forward_hook(self.save_gradient))

    def save_activation(self, module, input, output):
        activation = output
        self.activations.append(activation.cpu().detach())

    def save_gradient(self, module, input, output):
        if not hasattr(output, "requires_grad") or not output.requires_grad:
            # You can only register hooks on tensor requires grad.
            return

        # Gradients are computed in reverse order
        def _store_grad(grad):
            self.gradients = [grad.cpu().detach()] + self.gradients

        output.register_hook(_store_grad)

    def __call__(self, x):
        self.gradients = []
        self.activations = []
        return self.model(x)

    def release(self):
        for handle in self.handles:
            handle.remove()


In [49]:
def save_activation(module, input, output):
    global activations
    activation = output
    activations.append(activation.cpu().detach())

def save_gradient(module, input, output):
    if not hasattr(output, "requires_grad") or not output.requires_grad:
        # You can only register hooks on tensor requires grad.
        return

    # Gradients are computed in reverse order
    def _store_grad(grad):
        gradients = [grad.cpu().detach()] + gradients

    output.register_hook(_store_grad)

In [41]:
## instance input
model = cnn
target_layers = [cnn.conv_layers[2], cnn.conv_layers[6]]

## init
gradients = []
activations = []
handles = []
for target_layer in target_layers:
    handles.append(target_layer.register_forward_hook(save_activation))
    handles.append(target_layer.register_forward_hook(save_gradient))

In [42]:
## call
cnn(batch[0])

TypeError: save_activation() missing 1 required positional argument: 'output'

In [ ]:
def __call__(self, x):
    self.gradients = []
    self.activations = []
    return self.model(x)

def release(self):
    for handle in self.handles:
        handle.remove()

In [ ]:
for target_layer in target_layers:
            self.handles.append(
                target_layer.register_forward_hook(self.save_activation))

In [21]:
output = cnn(batch[0])

ReLU()

ReLU()

In [ ]:
activations_and_grads = ActivationsAndGradients(cnn, [cnn.conv_layers[2], cnn.conv_layers[6]])

In [ ]:
class ActivationsAndGradients:
    """ Class for extracting activations and
    registering gradients from targetted intermediate layers """

    def __init__(self, model, target_layers):
        self.model = model
        self.gradients = []
        self.activations = []
        self.handles = []
        for target_layer in target_layers:
            self.handles.append(
                target_layer.register_forward_hook(self.save_activation))
            # Because of https://github.com/pytorch/pytorch/issues/61519,
            # we don't use backward hook to record gradients.
            self.handles.append(
                target_layer.register_forward_hook(self.save_gradient))

    def save_activation(self, module, input, output):
        activation = output
        self.activations.append(activation.cpu().detach())

    def save_gradient(self, module, input, output):
        if not hasattr(output, "requires_grad") or not output.requires_grad:
            # You can only register hooks on tensor requires grad.
            return

        # Gradients are computed in reverse order
        def _store_grad(grad):
            self.gradients = [grad.cpu().detach()] + self.gradients

        output.register_hook(_store_grad)

    def __call__(self, x):
        self.gradients = []
        self.activations = []
        return self.model(x)

    def release(self):
        for handle in self.handles:
            handle.remove()


In [ ]:
# backward + 

# GRAD CAM